# Sediment Size Analysis by Sieve (SedSAS) Class : Example

### Processing sediment data stored in a comma-separated values (csv) text file:

## Part I: Introduction:

SedSAS is a class object written in the Python programming (scripting) language. Its purpose is to provide a basic set of statistical and visualization tools for analyzing unconsolidated sediment size-fraction samples collected in the field and separated using either mechanical sieves or any other analog partition-by-size methods. 

#### Multi-Sample Processing:

In this notebook we look at how the class is used to computer grain-size statistics for a one or more samples, where subsample weights are contained a separate comma-delineated text file. The data from that file will be read into this notebook and processed.

You can use this notebook not only as a learning tool or reference but also as a template from which to directly conduct your own analyses. To do the latter, simply download a copy and replace the existing data file path(s) and name(s), then run.

The downloaded notebook can be further modified, or copied and then the copy modified, as you see fit. 

#### What we'll do...

1.) Load data from a comma-separated-values (.csv) file into a Pandas dataframe <br/>
2.) Rework the contents of the new Pandas dataframe so as to meet the input and format requirements of SedSAS <br/>
3.) Compute grain-size statistics <br/>
4.) Write the results to a new Pandas dataframe and display the contents in the notebook <br/>
5.) Save the contents of the new results dataframe to a new csv file <br/>

From here you could further analyze the results data directly from the results dataframe, or import the output csv to R, a spreadsheet, or another analysis environment of your choosing.

### Preliminaries:
- The SedSAS module, this notebook, and all supporting materials were developed in a Python 3.x environment. Neither the module nor any of these supporting materials have been tested using a Python 2.x distro. *Backward compatibility is expected, but not assured.*
    
- If you do not have a Python interpreter installed on your computer (from the factory Linux UNIX, and MacOS users do; Windows users likely do not), your best bet for a trouble-free installation experience can be had here: https://anaconda.org/anaconda/python. If you're more adventurous, try the Python Software Foundation: https://www.python.org/getit/. Even if your OS does come with a Python interpreter already in place, it's probably a good idea to get a more up to date release (most OS installs are a version, or more, behind the current release). Again, I suggest the Anaconda distribution. Be sure to get a copy of Python version 3.5 or later.  
- Required external Python Libraries and modules\*\*:
    - sys (basic Python methods for retrieving information from the host operating system)
    - numpy (numeric Python library for array and matrix operations)
    - pandas (Python Data Analysis library for in-memory data storage and analysis)
    - matlotlib (Matrix Plotting Library for plot generation) 
    
    
- Loading SedSAS.py into your script: to use the class it must first be loaded into your script or notebook environment. If the class script file is located in the same directory as your analysis script then to load it you need only do the following:
    
    import  sedSAS
    
If the SedSAS.py file is located elsewhere on your computer or server you'll have to point Python to its location by amending the local path variable\*\*\*:

    import sys
    sys.path.append(/full/path/to/directory/where/class/file/is/located/)
    
For example, if your copy of SedSAS.py is located in the directory: 

    /Users/Documents/projects/ 
    
enter this into the sys.path.append method as:

    import sys
    sys.path.append(/Users/Documents/projects/)


______________
\*\*Note that these, and many, many other libraries, are included in the default Anaconda Python distribution.

\*\*\* Note that you are not changing your operating system's global PATH variable, only a copy of it that is assigned to the environment tied to your script or notebook. The global PATH variable is not altered

#### Required User Inputs:
Initial data input to SedSAS during the instantiation process consists of:

1. a listing of all the sieve apertures (in $\phi$ units) used in the analysis (sorted descending by size)
2. the weight of sediment material captured by each sieve 

These data must be passed to SedSAS inside a Pandas dataframe where the first column contains the aperture sizes in order as in the actual stack and where the second column contains the sediment weights.

A unique identifier for the sample can optionally be passed to SedSAS at the time of instantiation. Note that SedSAS doesn't really care if you provide a unique identifier, nor is it particular about how you choose to format the id. The identifier is more for the user and tracking than for code execution and so from a functional point of view, it's not even a requiremment. Nevertheless, it makes sense to provide something that helps to track what's in process, especially if you want to distinguish between samples when multiple samples are run in succession. If you wish to forego an id, the class will assign the default value: '1' as id each time the the class is instantiated.

## Part II: Example: using the class:

#### Computing grain-size statistics from sample data housed in a comma-delineated text file located in the subdirectory ./Sample_Data/. 

- The file to be processed is: DrySieveResults03.csv

- The sieves used (in stack order) in this example are: -1.0$\phi$, -0.5$\phi$, 0.0$\phi$, 0.5$\phi$, 1.0$\phi$, 1.25$\phi$, 1.5$\phi$, 1.75$\phi$, 2.0$\phi$, 2.5$\phi$, 3.0$\phi$, 3.5$\phi$, and 4.0$\phi$

- These apertures are not included as a data column in the file DrySieveResults03.csv, so we'll have to add them at the beginning of the analysis...

- The SedSAS.py script file is located in the same directory as is my (this) Jupyter notebook.

- Here are some of the contents of the file DrySieveResults03.csv:

Position,Pan_wt,Dry_wt,phi_n225,phi_n1,phi_n05,phi_00,phi_05,phi_10,phi_15,phi_20,phi_25,phi_30,phi_35,phi_40,dryfines </br>
0,,,0.86,0.28,0.26,0.36,0.83,1.79,5.17,12.06,13.88,2.09,0.27,0.27,0.02</br>
1,,,2.71,2.15,0.78,0.57,0.58,1.31,2.67,6.12,9.76,9.05,1.26,0.3,0.01</br>
2,,,0,0,0,0,0,0.05,0.41,0.41,4.19,19.02,7.25,1.31,0.16</br>
3,,,,,,,,,,,,,,,</br>
4,,,2.47,3.87,2.73,2.2,2.09,3.32,6.57,7.25,5.46,2.31,0.68,0.58,0.26</br>

Note that there is much more data (more data columns) than are needed/allowed for SedSAS. Once we load the data into a Pandas dataframe we'll have to delete these 'illegal' columns. We'll also add in new column for the sieve apertures and do one or two other things to get the data, and dataframe, in a form acceptable to SedSAS. You will probably have to perform similar 'data wrangling' tasks in order to prepare your own analyses. If you should elect to use SedSAS in your work just remember that Google can be your good friend in finding out how to do something with Python, numpy, and/or Pandas dataframes. Don't be afraid to experiment, try, and learn. 

Also, you'll note that there is a missing record (#3) in the data. That's okay. SedSAS has a means to gracefully detect and bypass missing data.

Finally, SedSAS is verbose. As you will see, during execution there is potentially going to be a lot of feedback written back to the console (your computer screen). You are advised to at least scan through this, voluminous as it might be, for important warnings, errors, and other information that might have a direct bearing on the integrity of the results. 

With all that said, let's get started...

### Read the contents of the CSV text file into a pandas dataframe and reformat as needed for SedSAS ingest:

In this particular example some columns from the source file are dropped, it was neceaary to transpose the new dataframe to arrange it so that each column represented a single sample and each row a single observation in a sample. We also updated the index. For other files, with other data content and formatting a different preparation will be required. The objective, however, is a simple dataframe containing one column that lists all the sieve screen apertures used in your analysis, and another second column that carries the individual subsample weights stopped by each concomitant sieve screen.


In [ ]:
import pandas as pd      
import numpy as np
import sys
import SedSAS

## list of sieve apertures used in the analysis. We'll use these as dataframe index values
Apertures=[-1.0,-0.5,0.0,0.5,1.0,1.25,1.5,1.75,2.0,2.5,3.0,3.5,4.0 ]

## First, read the raw file into a pandas dataframe:
file_path='../Sample_Data/'
file='DrySieveResults03.csv'
df=pd.read_csv(file_path+file)


## Then, we'll delete the Position, Pan_wt, and Dry_wt columns:
df=df.drop(['Position','Pan_wt','Dry_wt'], axis=1).copy()

## Then, then, we'll transpose the dataframe to change it from sample row into sample columnar order
df=df.T.copy()

## Finally, we'll reindex the data frame, inserting the sieve aperture values into df in place of the existing labels:
df.index=Apertures


## each column in df is a single sample. each row in df is a single observation in the sample

## Dataframe's ready to go, so let do the analysis...

### Compute grain-size statistical metrics for each sample in the dataframe

Iterate through the new dataframe, column by column (sample by sample), passing the sample to SedSAS for processing. Note that SedSAS is designed to handle only a single sample at a time. SedSAS analysis results for each and all samples in the input data file are returned in a new dataframe. This new dataframe can be the basis for continued analysis, or can be written out to a comma-separated values (csv) file--see "Save results to a Comma-Separated Values (csv) text file" at the end of this notebook for the details--for archive or in use eslewhere. 

This script uses a convenience method ComputeGSStats() to generate statistics (mean, sorting, skewness, and kurtosis) using 4 different computational approaches. Included in these appraoches are the logarithmic and geometric inclusive graphics from Folk and Ward (1957) and Folk, (1980) and the geomtric and logartihmic method of moments from Krumbien and Pettijohn,(1938). A fifth approach is represented in SedSAS, computation of the arithmetic method of moments, but as this is considered a much less robust and much less commonly used algorithm, it is omitted from ComputeGSStats(). If desired, the user can alter ComputeGSStats() to include this fifth method. 

In addition to ComputeGSStats() the user has the option to call methods which compute statistics using the five approaches represented in SedSAS (four of which are built into ComputeGSStats()) individually to customize analyses based on user need and experimental design. See the GitHub readme for more information.

Plots (histograms and CDFs) can also be generated for each sample and captured in individual (by sample) Portable Network Graphics (PNG) files. To do this add the following line of PLOTting code inside the for loop struct:    
    
    ## create a class instance and generate statistics using the convenience method ComputeGSStats():         <-- existing line
    out_list.append( ssc.ComputeGSStats() )                                                                   <-- existing line
    ssc.PLOTDualSampleWeightPercents(printTo='file')                                                          <-- add this line

In [ ]:
## set a counter variable to be used to create and pass a unique identifier to SedSAS for each sample
c=0

out_list=[]        # a temporary Python list to hold the dictionary of results returned for each sample in df

## loop thru the dataframe df, reading it column by column (by sample) and creating dataframe df_
## anew with each iteration. df_ will contain, for each iteration, an aperture column and a weight
## column. The latter is unique to the sample. This is passed to the SedSAS initializer.

for sample in df:
    df_= df[sample].to_frame()    # convert the single column from a series to a dataframe called df_
    df_.columns=['Weight']        # label the single column 'Weight'
    df_['Aperture']=df_.index     # copy the dataframe index to a new column 'Aperture'
    df_.set_index( [np.arange(0,len(df_['Aperture']),1)], inplace=True  )  # reset index to sequential numbers
    
    ID = 'Sample '+str(c)          # create an arbitrary unique idenifier for the current sample
    c=c+1
    
    ## create a class instance and generate statistics using the convenience method ComputeGSStats().
    ## ComputeGSStats() returns results in a Python dictionary that is then appended onto out_list
    ssc = SedSAS.SedSAS(df_, ID )
    out_list.append( ssc.ComputeGSStats() )
 
## create the output dataframe and write to screen:
df_out=pd.DataFrame(out_list).replace(np.nan, value='-')
df_out

#### Save results to a Comma-Separated Values (csv) text file:

In [ ]:
## saves the contents of the df_out dataframe to a csv file:
df_out.to_csv('./multi_sample_results.csv')